In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/final main.csv")
df.shape

(66397, 2)

In [ ]:

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [ ]:
df['desc'] = df['desc'].astype(str)

In [ ]:
df['cleaned'] = df['desc'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)


In [ ]:
df.to_csv("/content/drive/My Drive/finalDataCleaned.csv")

In [ ]:
df=pd.read_csv("/content/drive/My Drive/finalDataCleaned.csv")

In [ ]:
# # Downloading the Google pretrained Word2Vec Model
!wget -P "/content/drive/My Drive/" -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-07-12 05:26:27--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.144.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.144.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  43.2MB/s    in 37s     

2021-07-12 05:27:04 (42.2 MB/s) - ‘/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
EMBEDDING_FILE = '/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


In [ ]:
#splitting the description into words

i = 0
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [ ]:
# Training our corpus with Google Pretrained Model
%%time
google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(list_of_sentance)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 15s, sys: 630 ms, total: 1min 16s
Wall time: 1min 16s


In [ ]:
google_model = Word2Vec.load("/content/drive/My Drive/pharmEasyProductsWord2Vec.model")
# google_model.load("/content/drive/My Drive/pharmEasyProductsWord2Vec.model")

In [ ]:
google_model.wv.most_similar(positive=["fever"])

[('fevers', 0.6539823412895203),
 ('mania', 0.5566962957382202),
 ('feverish', 0.5537868142127991),
 ('giddiness', 0.49206802248954773),
 ('conjunctivitis', 0.47061413526535034),
 ('pneumonia', 0.4555879831314087),
 ('jaundice', 0.4547288715839386),
 ('euphoria', 0.4541453421115875),
 ('diarrhea', 0.4443085789680481),
 ('indigestion', 0.4330584406852722)]

In [ ]:
google_model.wv.most_similar(positive=["haldi"])

[('besan', 0.6918732523918152),
 ('methi', 0.6773034334182739),
 ('dhania', 0.663796067237854),
 ('kesar', 0.6617475748062134),
 ('amla', 0.6529474258422852),
 ('mirch', 0.6475131511688232),
 ('sindoor', 0.6453619003295898),
 ('turmeric', 0.6445726752281189),
 ('elaichi', 0.6389597654342651),
 ('pudina', 0.6377332806587219)]

In [ ]:
google_model.wv.most_similar(positive=["sanitary"])

[('hygienic', 0.72813880443573),
 ('sanitation', 0.6805286407470703),
 ('hygiene', 0.6576169729232788),
 ('cleanliness', 0.5477406978607178),
 ('disinfection', 0.5476149320602417),
 ('toilets', 0.5049561858177185),
 ('hygienically', 0.5028374195098877),
 ('disinfecting', 0.47032758593559265),
 ('handwash', 0.46137285232543945),
 ('disinfect', 0.455425500869751)]

In [ ]:
google_model.wv.most_similar(positive=["honey"])

[('syrup', 0.6025190353393555),
 ('berry', 0.5947865843772888),
 ('almond', 0.5711919665336609),
 ('berries', 0.5597745776176453),
 ('chocolate', 0.5528981685638428),
 ('apricot', 0.5451957583427429),
 ('demerara', 0.5406525135040283),
 ('cranberries', 0.538276195526123),
 ('jellies', 0.5353303551673889),
 ('blueberry', 0.5352541208267212)]

In [ ]:
global word_embeddings
word_embeddings = []

for line in df['cleaned']:
    avgword2vec = None
    count = 0
    for word in line.split():
        if word in google_model.wv.vocab:
            count += 1
            if avgword2vec is None:
                avgword2vec = google_model[word]
            else:
                avgword2vec = avgword2vec + google_model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
        word_embeddings.append(avgword2vec)

In [ ]:

def vectors2(x):
    
    global input_embeddings
    input_embeddings = []


    avgword2vec = None
    count = 0
    for word in x.split():
        if word in google_model.wv.vocab:
            count += 1
            if avgword2vec is None:
                avgword2vec = google_model[word]
            else:
                avgword2vec = avgword2vec + google_model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
    input_embeddings.append(avgword2vec)
    return input_embeddings


In [ ]:
def Nmaxelements(list1, N):
    final_list = []
    index_=[]
    
    for i in range(0, N): 
        max1 = 0
        index=-1      
        for j in range(len(list1)):     
            if list1[j] > max1:
                max1 = list1[j];
                index=j  
        list1.remove(max1);
        index_.append(index)
        final_list.append(max1)
          
    return index_

In [ ]:
def recommend(str,N):
  cosine_similarities = cosine_similarity(word_embeddings,vectors2("fever"))
  score=[]
  for i in range(0,len(cosine_similarities)):
    score.append(cosine_similarities[i][0])
  index=Nmaxelements(score,N)
  for i in index:
    print(df["product_info"][i])
    print(df["desc"][i])
    print("----")



In [ ]:
recommend("cold cough",10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


GUAPHA AYURVEDA AMRITARISHTA
guapha-ayurveda-amritarishta-otc529271Guapha Ayurveda Amritarishta is an ayurvedic medicine used in the treatment of chronic fever, typhoid fever, infectious fever, cough, cold and other infectious diseases. It is natural immunity booster and helps the body to fight off bacterial as well as viral infections. Generally, it is used for debility after fever.3.9 5
----
BAIDYANATH SARIWADI BATI
otc baidyanath-sariwadi-bati-otc424699Baidyanath Sariwadi Bati is an ayurvedic herbal medicine for treating ear infections and other ear ailments such as hearing disability, ear inflammation, ear pain etc. A widely used medicine in Indian Ayurveda, it is very helpful for treating tinnitus. Key Ingredients: Sariva - hemidesmus indicus. Cinnamon – cinnamomum.
----
REPL DR ADVICE NO.98 TYPHORIL DROP
 repl dr. advice no.98 typhoril drop 339456Information about REPL Dr. Advice No.98 Typhoril Drop Indication: Typhus fever, Shipboard fever, Adynamic fevers, Chill, With rheumatic

In [ ]:

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5, stop_words='english')
tfidf.fit(df['cleaned'])


tfidf_list = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names() 

In [ ]:
# Building TF-IDF Word2Vec 

tfidf_vectors = []; 
line = 0;
for desc in corpus: 
    sent_vec = np.zeros(300) 
    weight_sum =0; 
    for word in desc: 
        if word in google_model.wv.vocab and word in tfidf_feature:
            vec = google_model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_vectors.append(sent_vec)
    line += 1